In [2]:
import torch.optim as optim
from tqdm import tqdm
import numpy as np
import torch
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from numpy import asarray
import tifffile as tiff
import torch.nn as nn
import torch.nn.functional as nnFn
import torch_geometric.nn as pyg_nn
from torch_geometric.data import Data
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss
from sklearn.manifold import TSNE
import random
from torch_geometric.nn import GCNConv
import copy

In [3]:
# Load CN features
fa_feature_path = "/home/snu/Downloads/Histogram_CN_FA_20bin_updated.npy"
Histogram_feature_CN_FA_array = np.load(fa_feature_path, allow_pickle=True)

# Load AD features
fa_feature_path = "/home/snu/Downloads/Histogram_AD_FA_20bin_updated.npy"
Histogram_feature_AD_FA_array = np.load(fa_feature_path, allow_pickle=True)

# Combine features and labels
X = np.vstack([Histogram_feature_CN_FA_array, Histogram_feature_AD_FA_array])
y = np.hstack([
    np.zeros(Histogram_feature_CN_FA_array.shape[0], dtype=np.int64),
    np.ones(Histogram_feature_AD_FA_array.shape[0], dtype=np.int64)
])

np.random.seed(42)
perm = np.random.permutation(X.shape[0])
X = X[perm]
y = y[perm]
num_nodes, num_feats = X.shape
print(f"Features: {X.shape}, Labels: {y.shape}")

Features: (223, 180), Labels: (223,)


In [4]:
class MLP(nn.Module):
    def __init__(self, inp_size, outp_size, hidden_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(inp_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.PReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_size, outp_size)
        )

    def forward(self, x):
        return self.net(x)

In [5]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, device, activ):
        super(GCNEncoder, self).__init__()
        self.device = device
        self.gcn1 = GCNConv(input_dim, hidden_dim)
        self.batchnorm = nn.BatchNorm1d(hidden_dim)
        self.dropout = nn.Dropout(0.3)
        self.mlp = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.gcn1(x, edge_index)
        x = self.dropout(x)
        x = self.batchnorm(x)
        logits = self.mlp(x)
        return logits

In [6]:
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_clusters, device, activ):
        super(GCN, self).__init__()
        self.device = device
        self.num_clusters = num_clusters
        self.cut = 0

        self.online_encoder = GCNEncoder(input_dim, hidden_dim, device, activ)

        activations = {
            "SELU": nnFn.selu,
            "SiLU": nnFn.silu,
            "GELU": nnFn.gelu,
            "RELU": nnFn.relu
        }
        self.act = activations.get(activ, nnFn.elu)

        self.online_predictor = MLP(hidden_dim, num_clusters, hidden_size=hidden_dim)

        # attach modularity loss
        self.loss = self.cut_loss

    def forward(self, data):
        x = self.online_encoder(data)
        logits = self.online_predictor(x)
        S = nnFn.softmax(logits, dim=1)
        return S, logits

    def cut_loss(self, A, S):
        S = nnFn.softmax(S, dim=1)
        A_pool = torch.matmul(torch.matmul(A, S).t(), S)
        num = torch.trace(A_pool)

        D = torch.diag(torch.sum(A, dim=-1))
        D_pooled = torch.matmul(torch.matmul(D, S).t(), S)
        den = torch.trace(D_pooled)
        mincut_loss = -(num / den)

        St_S = torch.matmul(S.t(), S)
        I_S = torch.eye(self.num_clusters, device=self.device)
        ortho_loss = torch.norm(St_S / torch.norm(St_S) - I_S / torch.norm(I_S))

        return mincut_loss + ortho_loss


In [7]:
def create_adj(F, cut, alpha=1):
    F_norm = F / np.linalg.norm(F, axis=1, keepdims=True)
    W = np.dot(F_norm, F_norm.T)
    if cut == 0:
        W = np.where(W >= alpha, 1, 0).astype(np.float32)
        W = (W / W.max()).astype(np.float32)
    else:
        W = W - (W.max() / alpha)
    return W

In [8]:
def load_data(adj, node_feats):
    node_feats = torch.from_numpy(node_feats)
    edge_index = torch.from_numpy(np.array(np.nonzero((adj > 0))))
    row, col = edge_index
    edge_weight = torch.from_numpy(adj[row, col])
    return node_feats, edge_index, edge_weight

In [9]:
features = X.astype(np.float32)
print(features.shape, features.dtype)

cut = 0
alpha = 0.8
device = 'cuda' if torch.cuda.is_available() else 'cpu'
feats_dim = 180
K = 2

W0 = create_adj(features, 0, alpha)
node_feats, edge_index, _ = load_data(W0, features)
data0 = Data(x=node_feats, edge_index=edge_index).to(device)
A1 = torch.from_numpy(W0).float().to(device)
print(data0)

(223, 180) float32
Data(x=[223, 180], edge_index=[2, 41689])


In [10]:
from torch.optim.lr_scheduler import StepLR
from torch.optim import AdamW

model = GCN(feats_dim, 256, K, device, "ELU").to(device)
optimizer = AdamW(model.parameters(), lr=0.0001, weight_decay=0.0001)
scheduler = StepLR(optimizer, step_size=200, gamma=0.5)

num_epochs = 5000

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    S, logits = model(data0)
    unsup_loss = model.loss(A1, logits)

    total_loss = unsup_loss
    total_loss.backward()
    optimizer.step()
    scheduler.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch} | Loss: {total_loss:.4f}")


Epoch 0 | Loss: -0.2358
Epoch 100 | Loss: -0.4864
Epoch 200 | Loss: -0.5083
Epoch 300 | Loss: -0.5133
Epoch 400 | Loss: -0.5224
Epoch 500 | Loss: -0.5250
Epoch 600 | Loss: -0.5264
Epoch 700 | Loss: -0.5278
Epoch 800 | Loss: -0.5289
Epoch 900 | Loss: -0.5271
Epoch 1000 | Loss: -0.5308
Epoch 1100 | Loss: -0.5286
Epoch 1200 | Loss: -0.5262
Epoch 1300 | Loss: -0.5283
Epoch 1400 | Loss: -0.5278
Epoch 1500 | Loss: -0.5301
Epoch 1600 | Loss: -0.5264
Epoch 1700 | Loss: -0.5275
Epoch 1800 | Loss: -0.5282
Epoch 1900 | Loss: -0.5308
Epoch 2000 | Loss: -0.5311
Epoch 2100 | Loss: -0.5264
Epoch 2200 | Loss: -0.5287
Epoch 2300 | Loss: -0.5296
Epoch 2400 | Loss: -0.5299
Epoch 2500 | Loss: -0.5314
Epoch 2600 | Loss: -0.5322
Epoch 2700 | Loss: -0.5295
Epoch 2800 | Loss: -0.5295
Epoch 2900 | Loss: -0.5329
Epoch 3000 | Loss: -0.5279
Epoch 3100 | Loss: -0.5307
Epoch 3200 | Loss: -0.5287
Epoch 3300 | Loss: -0.5292
Epoch 3400 | Loss: -0.5264
Epoch 3500 | Loss: -0.5295
Epoch 3600 | Loss: -0.5298
Epoch 3700 | 

In [11]:
model.eval()
with torch.no_grad():
    S, logits = model(data0)
    y_pred = torch.argmax(logits, dim=1).cpu().numpy()
    y_pred_proba = nnFn.softmax(logits, dim=1).cpu().numpy()


acc_score = accuracy_score(y, y_pred)
acc_score_inverted = accuracy_score(y, 1 - y_pred)

if acc_score_inverted > acc_score:
    acc_score = acc_score_inverted
    y_pred = 1 - y_pred

prec_score = precision_score(y, y_pred)
rec_score = recall_score(y, y_pred)
f1 = f1_score(y, y_pred)
log_loss_value = log_loss(y, y_pred_proba)

print("Accuracy:", acc_score)
print("Precision:", prec_score)
print("Recall:", rec_score)
print("F1:", f1)
print("Log Loss:", log_loss_value)

Accuracy: 0.7713004484304933
Precision: 0.6788990825688074
Recall: 0.8222222222222222
F1: 0.7437185929648241
Log Loss: 1.5141893779587627


In [ ]:
print(y_pred)

[0 0 0 1 0 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 1 0 0 1 1 1 1 0 0 0 0 0 0 1 1 0 0
 0 0 1 1 1 1 1 1 0 0 0 1 1 0 1 1 0 0 0 1 0 1 0 1 0 0 1 0 1 1 0 1 1 0 1 0 1
 1 0 0 0 0 1 0 1 1 0 1 1 0 0 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0
 1 0 1 1 1 1 1 0 0 0 1 0 1 1 0 1 0 0 0 1 1 0 0 1 0 0 1 0 0 0 1 1 0 0 1 0 0
 0 1 1 1 0 0 0 1 1 1 1 0 0 1 1 0 0 1 1 1 0 1 0 0 0 0 0 1 0 0 1 1 1 0 0 0 1
 0 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 0 0 1 1 1 1 0 0 0 1 0 0 0 0 1
 0]


In [12]:
from torch.optim.lr_scheduler import StepLR
from torch.optim import AdamW

results = []

for run_seed in range(10):
    print("\n================ Run", run_seed, "================")

    # Set seeds for reproducibility
    np.random.seed(run_seed)
    torch.manual_seed(run_seed)
    random.seed(run_seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(run_seed)

    # Shuffle features and labels
    perm = np.random.permutation(X.shape[0])
    features = X[perm].astype(np.float32)
    labels = y[perm]

    cut = 0
    alpha = 0.8
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    feats_dim = 180
    K = 2

    W0 = create_adj(features, cut, alpha)
    node_feats, edge_index, _ = load_data(W0, features)
    data0 = Data(x=node_feats, edge_index=edge_index).to(device)
    A1 = torch.from_numpy(W0).float().to(device)

    model = GCN(feats_dim, 256, K, device, "ELU").to(device)
    optimizer = AdamW(model.parameters(), lr=0.0001, weight_decay=0.0001)
    scheduler = StepLR(optimizer, step_size=200, gamma=0.5)

    num_epochs = 5000

    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()


        S, logits = model(data0)
        unsup_loss = model.loss(A1, logits)

        unsup_loss.backward()
        optimizer.step()
        scheduler.step()

        if epoch % 1000 == 0:
            print(f"Epoch {epoch} | Loss: {unsup_loss:.4f}")

    model.eval()
    with torch.no_grad():
        S, logits = model(data0)
        y_pred = torch.argmax(logits, dim=1).cpu().numpy()
        y_pred_proba = nnFn.softmax(logits, dim=1).cpu().numpy()

    acc_score = accuracy_score(labels, y_pred)
    acc_score_inverted = accuracy_score(labels, 1 - y_pred)

    if acc_score_inverted > acc_score:
        acc_score = acc_score_inverted
        y_pred = 1 - y_pred

    prec_score = precision_score(labels, y_pred)
    rec_score = recall_score(labels, y_pred)
    f1 = f1_score(labels, y_pred)
    log_loss_value = log_loss(labels, y_pred_proba)

    print("Accuracy:", acc_score, "Precision:", prec_score, "Recall:", rec_score, "F1:", f1)

    results.append({
        "seed": run_seed,
        "accuracy": acc_score,
        "precision": prec_score,
        "recall": rec_score,
        "f1": f1,
        "log_loss": log_loss_value
    })

accs = [r["accuracy"] for r in results]
precisions = [r["precision"] for r in results]
recalls = [r["recall"] for r in results]
f1s = [r["f1"] for r in results]

print("\n===== Final Results across 10 runs =====")
print("Accuracy: mean=", np.mean(accs), "std=", np.std(accs))
print("Precision: mean=", np.mean(precisions), "std=", np.std(precisions))
print("Recall: mean=", np.mean(recalls), "std=", np.std(recalls))
print("F1: mean=", np.mean(f1s), "std=", np.std(f1s))



================ Run 0 ================
Epoch 0 | Loss: -0.2389
Epoch 1000 | Loss: -0.5274
Epoch 2000 | Loss: -0.5305
Epoch 3000 | Loss: -0.5287
Epoch 4000 | Loss: -0.5300
Accuracy: 0.7668161434977578 Precision: 0.6759259259259259 Recall: 0.8111111111111111 F1: 0.7373737373737373

================ Run 1 ================
Epoch 0 | Loss: -0.2333
Epoch 1000 | Loss: -0.5253
Epoch 2000 | Loss: -0.5305
Epoch 3000 | Loss: -0.5282
Epoch 4000 | Loss: -0.5292
Accuracy: 0.7713004484304933 Precision: 0.6788990825688074 Recall: 0.8222222222222222 F1: 0.7437185929648241

================ Run 2 ================
Epoch 0 | Loss: -0.2350
Epoch 1000 | Loss: -0.5285
Epoch 2000 | Loss: -0.5287
Epoch 3000 | Loss: -0.5293
Epoch 4000 | Loss: -0.5292
Accuracy: 0.7713004484304933 Precision: 0.6788990825688074 Recall: 0.8222222222222222 F1: 0.7437185929648241

================ Run 3 ================
Epoch 0 | Loss: -0.2374
Epoch 1000 | Loss: -0.5206
Epoch 2000 | Loss: -0.5211
Epoch 3000 | Loss: -0.5233
Epoch 40